**CÀI ĐẶT VÀ IMPORT THƯ VIỆN**

In [ ]:
!pip install catalyst==20.2.4
!pip install tqdm==4.33
!pip install efficientnet_pytorch
!pip install pytorch_toolbelt

In [ ]:
from sklearn import svm
from keras.callbacks import CSVLogger
from keras.callbacks import LambdaCallback
from utils.utils import *
import pandas as pd
import sys
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import torch
import collections
from pytorch_toolbelt.inference import tta
from catalyst.dl.callbacks import InferCallback
from catalyst.dl.runner import SupervisedRunner
from torch.nn.functional import softmax
from catalyst.dl.callbacks import EarlyStoppingCallback, AccuracyCallback, F1ScoreCallback, ConfusionMatrixCallback, MixupCallback
from catalyst.contrib.nn.schedulers import OneCycleLR, ReduceLROnPlateau, StepLR, MultiStepLR
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
import pickle
import time
import copy
import sklearn

In [ ]:
data = pd.read_csv('./utils/splited_train_new.csv')
splits = pickle.load(open('./utils/cv_split.pickle', 'rb'))
labels = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
image_size = 240
fold_idx = 3
batch_size = 1
lr = 1e-5

**TRAIN**

In [ ]:
train_path1 = './data/clahe-train/'
valid_path1 = './data/clahe-train/'

train_dataset1 = EyeDataset(dataset_path=train_path1,
                           labels=data.loc[splits['train_idx'][fold_idx],labels].values,
                           ids=data.loc[splits['train_idx'][fold_idx], 'id'].values,
                           albumentations_tr=aug_train_heavy(image_size))

train_loader1 = DataLoader(train_dataset1,
                          num_workers=8,
                          pin_memory=False,
                          batch_size=batch_size,
                          shuffle=False)


logdir = './logs/'
modelA = prepare_model('efficientnet-b3', 8)
modelA.cuda()
modelA.load_state_dict(torch.load(os.path.join(logdir, './logs/clahe.pth'))['model_state_dict'])
modelA.eval()

In [ ]:
train_path2 = './data/vessel-train/'
valid_path2 = './data/vessel-train/'

train_dataset2 = EyeDataset(dataset_path=train_path2,
                           labels=data.loc[splits['train_idx'][fold_idx], labels].values,
                           ids=data.loc[splits['train_idx'][fold_idx], 'id'].values,
                           albumentations_tr=aug_train_heavy(image_size))

train_loader2 = DataLoader(train_dataset2,
                          num_workers=8,
                          pin_memory=False,
                          batch_size=batch_size,
                          shuffle=False)

logdir = './logs/'
modelB = prepare_model('efficientnet-b3', 8)
modelB.cuda()
modelB.load_state_dict(torch.load(os.path.join(logdir, './logs/vessel.pth'))['model_state_dict'])
modelB.eval()

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, modelA, modelB, nb_classes=8):
        super(NeuralNet, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.drop = nn.Dropout(0.3)
        self.out2 = nn.Linear(1536, 8)
        self.modelA.fc = nn.Identity()
        self.modelB.fc = nn.Identity()

    def forward(self, x1, x2):
        x1 = self.modelA.extract_features(x1)
        x2 = self.modelB.extract_features(x2)
        
        x1 = nn.AdaptiveAvgPool2d((1, 1))(x1)
        x2 = nn.AdaptiveAvgPool2d((1, 1))(x2)
        x1 = x1.view(1,1536)
        x2 = x2.view(1,1536)
        x = x1+x2    
        sigmoid = torch.sigmoid(x)
        multi = torch.mul(sigmoid,x2)
        x = multi + x1
        x = self.drop(x)
        x = x.flatten()
        x = self.out2(x)
        return x


In [ ]:
x_model = NeuralNet(modelA, modelB)
x_model.eval()

In [ ]:
optimizer = torch.optim.Adam(x_model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.5, patience=5)

PATH_SAVE = './logs/'
if not os.path.exists(PATH_SAVE):
       os.makedirs(PATH_SAVE)
for epoch in range(20):
  for i1, i2 in zip(enumerate(train_loader1),enumerate(train_loader2)):
    dem = dem + 1
    stt1, data1 = i1
    input1,target1 = data1

    stt2, data2 = i2
    input2,target2 = data2

    optimizer.zero_grad()
    input1 = input1.cuda()
    input2 = input2.cuda()

    outputs = x_model(input1,input2)
    outputs = outputs.view(1,8)

    loss = criterion(outputs, target1)
    loss.backward()
    optimizer.step()
torch.save(x_model.state_dict(), PATH_SAVE + 'final.pth')

**TEST CONCATE CLAHE VESSEL**

In [ ]:
test_data = pd.read_csv('./utils/XYZ_ODIR.csv')
test_data_left = test_data.copy()
test_data_right = test_data.copy()
test_data_left.loc[:, 'id'] = test_data_left.ID.apply(
    lambda x: str(x)+'_left.jpg')
test_data_right.loc[:, 'id'] = test_data_left.ID.apply(
    lambda x: str(x)+'_right.jpg')
test_data = pd.concat([test_data_left, test_data_right])
test_data.sort_values(['ID'], inplace=True)

test_path1 = './data/clahe-test/'
test_dataset_clahe = EyeDataset(dataset_path=test_path1,
                                labels=test_data.loc[:, labels].values,
                                ids=test_data.loc[:, 'id'].values,
                                albumentations_tr=aug_val(image_size))
test_loader_clahe = DataLoader(test_dataset_clahe,
                                num_workers=8,
                                pin_memory=False,
                                batch_size=batch_size,
                                shuffle=False)

test_path2 = './data/vessel-test/'
test_dataset_vessel = EyeDataset(dataset_path=test_path2,
                                labels=test_data.loc[:, labels].values,
                                ids=test_data.loc[:, 'id'].values,
                                albumentations_tr=aug_val(image_size))
test_loader_vessel = DataLoader(test_dataset_vessel,
                                num_workers=8,
                                pin_memory=False,
                                batch_size=batch_size,
                                shuffle=False)

In [ ]:
x_model = NeuralNet(modelA,modelB)
x_model.load_state_dict(torch.load(os.path.join('./logs/final.pth')))
x_model.eval()
for i1, i2 in zip(enumerate(test_loader_clahe), enumerate(test_loader_vessel)):
    stt1, data1 = i1
    input1, target1 = data1
    stt2, data2 = i2
    input2, target2 = data2
    input1 = input1.cuda()
    input2 = input2.cuda()

    outputs = x_model(input1, input2)
    outputs = torch.Tensor.cpu(outputs).detach().numpy()
    probabilities = softmax(torch.from_numpy(outputs), dim=0).numpy()
    for idx in range(probabilities.shape[0]):
        if all(probabilities[:] < 0.5):
            probabilities[0] = 1.0
    probabilities_list.append(probabilities)
arr_list = []
arr = np.array(probabilities_list)
arr_list.append(arr)
probabilities_combined = np.stack(arr_list, axis=0).mean(axis=0)
predicted_labels = pd.DataFrame(probabilities_combined, columns=labels)
predicted_labels['id'] = test_data.loc[:, 'id'].values
predicted_labels.loc[:, 'ID'] = predicted_labels.id.apply(
    lambda x: x.split('_')[0])
predicted_labels_groupped = predicted_labels.groupby(
    ['ID']).aggregate(dict(zip(labels, ['max']*(len(labels)))))
print(type(predicted_labels_groupped))
predicted_labels_groupped['ID'] = predicted_labels_groupped.index.values.astype(
    int)
predicted_labels_groupped.reset_index(drop=True, inplace=True)
predicted_labels_groupped.sort_values('ID', inplace=True)
predicted_labels_groupped = predicted_labels_groupped.loc[:, ['ID']+labels]
predicted_labels_groupped.to_csv('./submit/final.csv', index=False)